In [1]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"

import util_funcs
from importlib import reload
import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import ensembleReader as er
import clinical_text_analysis as cta
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from os import path
# import predictSeizureConvExp as psce
import keras_models.dataGen as dg
from addict import Dict
from pathlib import Path
# reload(psce)
from keras.utils import multi_gpu_model
import keras
import keras.optimizers
import time
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, log_loss

Using TensorFlow backend.


In [2]:
train_data = pkl.load(open("/n/scratch2/ms994/train_multiple_labels_sessions_montage_seizure_data_4.pkl", "rb"))

In [28]:
test_data =  pkl.load(open("/n/scratch2/ms994/test_multiple_labels_sessions_montage_seizure_data_4.pkl", "rb"))

In [29]:
valid_data = pkl.load(open("/n/scratch2/ms994/valid_multiple_labels_sessions_montage_seizure_data_4.pkl", "rb"))

In [5]:
def patient_func(tkn_file_paths, session_instead_patient):
        return [read.parse_edf_token_path_structure(tkn_file_path)[1] + "/" + read.parse_edf_token_path_structure(tkn_file_path)[2] for tkn_file_path in tkn_file_paths]

In [6]:
seizureInfo = pd.read_csv("/home/ms994/dbmi_eeg_clustering/assets/seizures.csv")

In [7]:
seizureInfo.groupby("seizure_type")["seizure_type"].count() 

seizure_type
ABSZ      99
CPSZ     361
FNSZ    1834
GNSZ     585
MYSZ       3
SPSZ      52
TCSZ      53
TNSZ      68
Name: seizure_type, dtype: int64

Use the top 3 seizures only, CPSZ, FNSZ, and GNSZ. only include a patient if they are characterized by the seizure or have no seizure in the set. It is either unfair/incorrect/too hard to include the healthy state of patients not in this subpopulation

In [8]:
noSeizureFiles = seizureInfo[pd.isnull(seizureInfo.seizure_type)].fn

In [9]:
seizurePatientMap = seizureInfo.groupby(["patient", "seizure_type"])["patient", "seizure_type"].count()

In [10]:
seizurePatientMap.columns = ["patient_cnt", "seizure_type_cnt"]

In [11]:
seizurePatientMap = seizurePatientMap.reset_index()

In [12]:
matchingPatients = seizurePatientMap[seizurePatientMap.seizure_type.isin(["CPSZ", "FNSZ", "GNSZ"])].patient

In [13]:
noSeizurepatients = [read.parse_edf_token_path_structure(file)[1] for file in noSeizureFiles]

In [14]:
matchingPatients = matchingPatients.apply(lambda a: "{:08d}".format(int(a))).tolist()

In [15]:
toFilterBy = noSeizurepatients + matchingPatients

In [16]:
"{}/{}".format(read.parse_edf_token_path_structure(noSeizureFiles[0])[1], read.parse_edf_token_path_structure(noSeizureFiles[0])[2])

'00000258/s002_2003_07_21'

In [17]:
(train_data[0][0].std(axis=0) > 100).any()

False

In [18]:
def update_data(edss,  seizure_classes_to_use=["GNSZ", "FNSZ", "CPSZ"], include_seizure_type=True, include_montage_channels=True, remove_outlier_by_std_thresh=100, zero_center_over_time=True, is_eval_set=False):
    '''
    since we store the full string of the session or the patient instead of the index, we update the data to use the int index
    some of the tasks require different datasets and some filtering of the data i.e. only seizure classification or just some of the labels
    is_eval_set: for use with valid and test set, since they shouldn't have patients from the train set and therefore predicting for them is wrong
    '''

    data = [datum[0] for datum in edss]
    if is_eval_set:
        patient_labels = [0 for i in range(len(edss))]
        remove_outlier_by_std_thresh=None
    else:
        patients = [datum[1][1] for datum in edss]
        allPatient = list(set(patients))
        patient_labels = [allPatient.index(patient) for patient in patients]
    seizure_detection_labels = [datum[1][0] for datum in edss]
    if include_seizure_type:
        seizure_class_labels = [datum[1][2] for datum in edss]
    if include_montage_channels:
        montage_channel_labels = [datum[1][3] for datum in edss]
    keep_index = [True for i in range(len(data))]
    if seizure_classes_to_use is not None:
        for i, seizure_class_label in enumerate(seizure_class_labels):
            if seizure_detection_labels[i] and constants.SEIZURE_SUBTYPES[seizure_class_label] not in seizure_classes_to_use: #it's a seizure detection class and it should be excluded
                keep_index[i] = False
    for i, patient_session in enumerate(patients):
        if patient_session.split("/")[0] not in toFilterBy:
            keep_index[i] = False
        

    if zero_center_over_time:
        for i, datum in enumerate(data):
            data[i] = (datum.T - datum.mean(1)).T
    if remove_outlier_by_std_thresh is not None:
        removed = 0
        for i, datum in enumerate(data):
            if (np.std(datum, axis=0) > remove_outlier_by_std_thresh).any():
                keep_index[i] = False
                removed += 1
        print("We removed {} because unclean by std".format(removed))


    data_to_keep = []
    patient_labels_to_keep = []
    seizure_detect_to_keep = []
    seizure_class_labels_to_keep = []
    montage_channel_labels_to_keep = []

    for i, should_keep in enumerate(keep_index):
        if should_keep:
            data_to_keep.append(data[i])
            patient_labels_to_keep.append(patient_labels[i])
            seizure_detect_to_keep.append(seizure_detection_labels[i])
            if include_montage_channels:
                montage_channel_labels_to_keep.append(montage_channel_labels[i])
            if include_seizure_type:
                seizure_class_labels_to_keep.append(seizure_class_labels[i])
    return [(data_to_keep[i], (seizure_detect_to_keep[i], patient_labels_to_keep[i], seizure_class_labels_to_keep[i], montage_channel_labels_to_keep[i].values)) for i in range(len(data_to_keep))], seizure_detect_to_keep, patient_labels_to_keep, seizure_class_labels_to_keep, montage_channel_labels_to_keep

In [19]:
train_data = update_data(train_data)

We removed 32959 because unclean by std
We removed 4882 because unclean by std
We removed 10631 because unclean by std


In [26]:
train_data = train_data[0]

In [30]:
valid_data = update_data(valid_data, remove_outlier_by_std_thresh=None)
test_data = update_data(test_data, remove_outlier_by_std_thresh=None)

In [31]:
valid_data = valid_data[0]
test_data = test_data[0]

In [27]:
import tensorflow as tf
# Helperfunctions to make your feature definition more readable
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _int64_feature_list(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
# Helperfunctions to make your feature definition more readable
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _float_feature_list(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [35]:
valid_data[0][1][3].astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [36]:
is_train = True
def parse_datum(datum):
    global is_train
    xData = datum[0]
    yData = datum[1]
    
    feature = { \
               'data': _float_feature_list(xData.reshape(-1)), \
               'label': _int64_feature_list([yData[0]]), \
               'subtypeLabel': _int64_feature_list([yData[2]]), \
               'session': _int64_feature(yData[1]), \
               'montage': _int64_feature_list(yData[3].astype(int))
              }
    return tf.train.Example(features=tf.train.Features(feature=feature))


In [47]:
def get_gen(dataset):
    def gen():
        for i in range(len(dataset)):
            yield parse_datum(dataset[i]).SerializeToString()
    return gen

In [49]:
dataset = tf.data.Dataset.from_generator(get_gen(train_data), output_types=tf.string,  output_shapes=(tf.TensorShape([])))

In [50]:
dataset = tf.data.Dataset.from_generator(get_gen(train_data), output_types=tf.string,  output_shapes=(tf.TensorShape([])))
writer = tf.data.experimental.TFRecordWriter("/n/scratch2/ms994/train_4s.tfr")
writer.write(dataset)

In [51]:
dataset = tf.data.Dataset.from_generator(get_gen(valid_data), output_types=tf.string,  output_shapes=(tf.TensorShape([])))
writer = tf.data.experimental.TFRecordWriter("/n/scratch2/ms994/valid_4s.tfr")
writer.write(dataset)

In [52]:
dataset = tf.data.Dataset.from_generator(get_gen(test_data), output_types=tf.string,  output_shapes=(tf.TensorShape([])))
writer = tf.data.experimental.TFRecordWriter("/n/scratch2/ms994/test_4s.tfr")
writer.write(dataset)